In [1]:
!pip install google-search-results numexpr --quiet
!pip install -q langgraph pgvector langchain pypdf sentence-transformers psycopg langchain-community langchain-anthropic tavily-python pandas openai --quiet
# !pip install tavily-python 
!pip install -U tavily-python langchain_community sentence-transformers --quiet
!pip install autogen-agentchat~=0.2 autogen psutil --quiet
# !pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client==0.12.3 langgraph langchain_experimental yfinance
!pip install -q langchain-openai langchain-anthropic termcolor langchain_community duckduckgo_search wikipedia openapi-python-client langgraph langchain_experimental openai --quiet



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# 添加Langsmith 的Tracking
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Chapter06_ReAct_Handson_AI_Agent"
os.environ["LANGSMITH_API_KEY"] = "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"

In [3]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

os.environ["TAVILY_API_KEY"] = "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"
os.environ['ATHINA_API_KEY'] = "IhrJrr0krTMRA9ogqi5aaD4ZuYuvMcdG"

os.environ['SERPAPI_API_KEY']= "d79f01166cee047474b994092fd5f2462c2461490acf51ad91bf48ef10f46c28"


INFERENCE_SERVER_URL = "http://localhost:8989"
# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
API_KEY= "alanliuxiang"

llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


In [4]:
# from langchain_community.tools.tavily_search import TavilySearchResults
# tool = TavilySearchResults(max_results=2)
# tools = [tool]
# tool.invoke("What's a 'node' in LangGraph?")



In [5]:
# 设置工具
# from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
# from langchain_community.utilities import SerpAPIWrapper
# from langchain.agents import load_tools
# tools = load_tools(["serpapi", "llm-math"], 
#                    llm=llm)

In [6]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain_community.utilities import SerpAPIWrapper

search = SerpAPIWrapper()
tools = [
    Tool(
        name="llm-math",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
]


In [7]:
# 设置提示模板
from langchain.prompts import PromptTemplate
template = ('''
    '尽你所能用中文回答以下问题。如果能力不够你可以使用以下工具:\n\n'
    '{tools}\n\n
    Use the following format:\n\n'
    'Question: the input question you must answer\n'
    'Thought: you should always think about what to do\n'
    'Action: the action to take, should be one of [{tool_names}]\n'
    'Action Input: the input to the action\n'
    'Observation: the result of the action\n'
    '... (this Thought/Action/Action Input/Observation can repeat N times)\n'
    'Thought: I now know the final answer\n'
    'Final Answer: the final answer to the original input question\n\n'
    'Begin!\n\n'
    'Question: {input}\n'
    'Thought:{agent_scratchpad}' 
    '''
)
prompt = PromptTemplate.from_template(template)

In [8]:
# 初始化Agent
from langchain.agents import create_react_agent
agent = create_react_agent(llm,
                           tools,
                           prompt)

# 构建AgentExecutor
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               handle_parsing_errors=True,
                               verbose=True)

# 执行AgentExecutor
agent_executor.invoke({"input": 
                       """目前市场上玫瑰花的一般进货价格是多少？\n
                       如果我在此基础上加价5%，应该如何定价？"""})



> Entering new AgentExecutor chain...
整。
</think>

为了确定玫瑰花的进货价格和加价5%后的定价，我需要先了解当前的市场行情。由于无法直接访问实时数据，我将基于一般市场情况估算。

假设普通玫瑰花的进货价格大约是每枝3元。加价5%后，计算如下：

3元 × 1.05 = 3.15元

因此，建议将玫瑰花的零售价定为每枝3.15元左右。

Final Answer: 建议将玫瑰花的零售价定为每枝3.15元左右。整。
</think>

为了确定玫瑰花的进货价格和加价5%后的定价，我需要先了解当前的市场行情。由于无法直接访问实时数据，我将基于一般市场情况估算。

假设普通玫瑰花的进货价格大约是每枝3元。加价5%后，计算如下：

3元 × 1.05 = 3.15元

因此，建议将玫瑰花的零售价定为每枝3.15元左右。

Final Answer: 建议将玫瑰花的零售价定为每枝3.15元左右。

> Finished chain.


{'input': '目前市场上玫瑰花的一般进货价格是多少？\n\n                       如果我在此基础上加价5%，应该如何定价？',
 'output': '建议将玫瑰花的零售价定为每枝3.15元左右。'}